# Segmenting and Clustering Neighborhoods in Toronto

## Table of contents
* [Part 1](#section1)
* [Part 2](#section2)
* [Part 3](#section3)

<a id="section1"></a>
## Part 1 - Scraping the data

In [2]:
#Import
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup # this module helps in web scrapping.

In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(URL).text
soup = BeautifulSoup(data,"html5lib")

#find all html tables in the web page
tables = soup.find_all('table') # in html table is represented by the tag <table>
print(f"There are {len(tables)} tables on this website.")

# target table is table 0
targetTable = tables[0]
#print(tables[0].prettify())

There are 3 tables on this website.


In [4]:
# Creating Dataframe from scraped data
data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

for row in targetTable.tbody.find_all("tr"):
    fields = row.find_all("td")
    
    for field in fields:
        postalCode = field.find("b").text
        if field.find("span").text == "Not assigned":
            continue
        borough = field.find("span").text.split("(")[0]
        neighborhood = field.find("span").text.split("(")[1]
        neighborhood = neighborhood.split(")")[0]
        
        # if neighborhoods are separeted by "/", replace by delimiter ","
        if "/" in neighborhood:
            neighborhood = neighborhood.replace("/", ",")
        
        data = data.append({"PostalCode":postalCode,
                            "Borough":borough,
                            "Neighborhood":neighborhood}, ignore_index=True)

In [5]:
data.shape

(103, 3)

<a id="section2"></a>
# Part 2 - Using API to get Latitude and Longitude

In [6]:
geodata = pd.read_csv("Geospatial_Coordinates.csv")
geodata.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [7]:
data.head()

PostalCode           Borough                       Neighborhood
0        M3A        North York                          Parkwoods
1        M4A        North York                   Victoria Village
2        M5A  Downtown Toronto         Regent Park , Harbourfront
3        M6A        North York  Lawrence Manor , Lawrence Heights
4        M7A      Queen's Park      Ontario Provincial Government

In [8]:
lat = []
long = []

for postalCode in data['PostalCode']:
    lat.append(geodata[geodata['Postal Code'] == postalCode]["Latitude"].values[0])
    long.append(geodata[geodata['Postal Code'] == postalCode]["Longitude"].values[0])
    

In [9]:
data['Latitude'] = lat
data['Longitude'] = long

In [10]:
data.head()

PostalCode           Borough                       Neighborhood   Latitude  \
0        M3A        North York                          Parkwoods  43.753259   
1        M4A        North York                   Victoria Village  43.725882   
2        M5A  Downtown Toronto         Regent Park , Harbourfront  43.654260   
3        M6A        North York  Lawrence Manor , Lawrence Heights  43.718518   
4        M7A      Queen's Park      Ontario Provincial Government  43.662301   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494

<a id="section3"></a>
# Part 3 - Exploring and clustering the data

In [11]:
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [12]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [13]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
toronto_data = data[data['Borough'].str.contains("Toronto")].reset_index(drop=True)

In [15]:
toronto_data.head()

PostalCode           Borough                Neighborhood   Latitude  \
0        M5A  Downtown Toronto  Regent Park , Harbourfront  43.654260   
1        M5B  Downtown Toronto    Garden District, Ryerson  43.657162   
2        M5C  Downtown Toronto              St. James Town  43.651494   
3        M4E      East Toronto                 The Beaches  43.676357   
4        M5E  Downtown Toronto                 Berczy Park  43.644771   

   Longitude  
0 -79.360636  
1 -79.378937  
2 -79.375418  
3 -79.293031  
4 -79.373306

In [16]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [17]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [18]:
with open("./Credentials/client_ID.txt") as file:
    CLIENT_ID = file.read() # your Foursquare ID
with open("./Credentials/client_secret.txt") as file:
    CLIENT_SECRET = file.read() # your Foursquare Secret 
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park , Harbourfront
429


KeyError: 'groups'

In [20]:
toronto_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park , Harbourfront               43.65426              -79.360636   
1  Regent Park , Harbourfront               43.65426              -79.360636   
2  Regent Park , Harbourfront               43.65426              -79.360636   
3  Regent Park , Harbourfront               43.65426              -79.360636   
4  Regent Park , Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0           Tandem Coffee       43.653559       -79.361809   
1        Roselle Desserts       43.653447       -79.362017   
2  Cooper Koo Family YMCA       43.653249       -79.358008   
3          Impact Kitchen       43.656369       -79.356980   
4     Body Blitz Spa East       43.654735       -79.359874   

        Venue Category  
0          Coffee Shop  
1               Bakery  
2  Distribution Center  
3           Restaurant  
4                  Spa

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Airport  Airport Food Court  Airport Lounge  Airport Service  \
0            0        0                   0               0                0   
1            0        0                   0               0                0   
2            0        0                   0               0                0   
3            0        0                   0               0                0   
4            0        0                   0               0                0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0                 0                    0             0         0            0   
1                 0                    0             0         0            0   
2                 0                    0             0         0            0   
3                 0                    0             0         0            0   
4                 0                    0             0         0            0   

   ...  Tibetan Restaurant  Toy / Game Store  Trail  Train Station  \
0  ...                   0                 0      0              0   
1  ...                   0                 0      0              0   
2  ...                   0                 0      0              0   
3  ...                   0                 0      0              0   
4  ...                   0                 0      0              0   

   Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                              0                 0                      0   
1                              0                 0                      0   
2                              0                 0                      0   
3                              0                 0                      0   
4                              0                 0                      0   

   Wine Bar  Wings Joint  Women's Store  
0         0            0              0  
1         0            0              0  
2         0            0              0  
3         0            0              0  
4         0            0              0  

[5 rows x 230 columns]

In [1]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

NameError: name 'toronto_onehot' is not defined

In [23]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.08
1          Bakery  0.05
2    Cocktail Bar  0.05
3  Farmers Market  0.03
4        Beer Bar  0.03


----Brockton , Parkdale Village , Exhibition Place----
                   venue  freq
0                   Café  0.12
1         Breakfast Spot  0.08
2  Performing Arts Venue  0.08
3            Coffee Shop  0.08
4                 Bakery  0.08


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3       Coffee Shop  0.07
4           Airport  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1      Sandwich Place  0.06
2                Café  0.05
3  Italian Restaurant  0.05
4     Thai Restaurant  0.03


----Christie----
           venue  freq
0  Grocery Store  0.24
1           Café  0.18
2           Park  0.12
3     

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1     Brockton , Parkdale Village , Exhibition Place                  Café   
2  CN Tower , King and Spadina , Railway Lands , ...        Airport Lounge   
3                                 Central Bay Street           Coffee Shop   
4                                           Christie         Grocery Store   

  2nd Most Common Venue  3rd Most Common Venue 4th Most Common Venue  \
0                Bakery           Cocktail Bar        Farmers Market   
1        Breakfast Spot  Performing Arts Venue           Coffee Shop   
2       Airport Service       Airport Terminal           Coffee Shop   
3        Sandwich Place                   Café    Italian Restaurant   
4                  Café                   Park            Baby Store   

  5th Most Common Venue 6th Most Common Venue   7th Most Common Venue  \
0              Beer Bar              Pharmacy             Cheese Shop   
1                Bakery           Yoga Studio  Furniture / Home Store   
2               Airport      Sculpture Garden     Rental Car Location   
3       Thai Restaurant       Bubble Tea Shop            Burger Joint   
4           Coffee Shop             Nightclub      Italian Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0            Restaurant    Seafood Restaurant      Indian Restaurant  
1         Burrito Place          Climbing Gym      Convenience Store  
2         Boat or Ferry              Boutique                  Plane  
3           Salad Place                   Spa      French Restaurant  
4           Candy Store    Athletics & Sports             Restaurant

### Cluster neighborhood

In [26]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [27]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the first columns!

PostalCode           Borough                Neighborhood   Latitude  \
0        M5A  Downtown Toronto  Regent Park , Harbourfront  43.654260   
1        M5B  Downtown Toronto    Garden District, Ryerson  43.657162   
2        M5C  Downtown Toronto              St. James Town  43.651494   
3        M4E      East Toronto                 The Beaches  43.676357   
4        M5E  Downtown Toronto                 Berczy Park  43.644771   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.360636               0           Coffee Shop                Bakery   
1 -79.378937               0        Clothing Store           Coffee Shop   
2 -79.375418               0                  Café           Coffee Shop   
3 -79.293031               0     Health Food Store                 Trail   
4 -79.373306               0           Coffee Shop                Bakery   

  3rd Most Common Venue 4th Most Common Venue      5th Most Common Venue  \
0                  Park                   Pub             Breakfast Spot   
1        Cosmetics Shop                  Café  Middle Eastern Restaurant   
2          Cocktail Bar        Cosmetics Shop        Moroccan Restaurant   
3                   Pub           Yoga Studio                     Museum   
4          Cocktail Bar        Farmers Market                   Beer Bar   

      6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0                      Café               Theater        Cosmetics Shop   
1           Bubble Tea Shop      Ramen Restaurant          Burger Joint   
2        Seafood Restaurant              Creperie            Restaurant   
3  Mediterranean Restaurant           Men's Store         Metro Station   
4                  Pharmacy           Cheese Shop            Restaurant   

  9th Most Common Venue     10th Most Common Venue  
0                   Spa                 Shoe Store  
1        Lingerie Store                  Bookstore  
2    Italian Restaurant           Department Store  
3    Mexican Restaurant  Middle Eastern Restaurant  
4    Seafood Restaurant          Indian Restaurant

In [29]:
print(f"There are {len(toronto_merged['Cluster Labels'].unique())} different clusters in the dataset.")

There are 5 different clusters in the dataset.


In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the clusters

### Cluster 1

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels  \
0                                    Downtown Toronto               0   
1                                    Downtown Toronto               0   
2                                    Downtown Toronto               0   
3                                        East Toronto               0   
4                                    Downtown Toronto               0   
5                                    Downtown Toronto               0   
6                                    Downtown Toronto               0   
7                                    Downtown Toronto               0   
8                                        West Toronto               0   
10                                   Downtown Toronto               0   
11                                       West Toronto               0   
12                                       East Toronto               0   
13                                   Downtown Toronto               0   
14                                       West Toronto               0   
15                                       East Toronto               0   
16                                   Downtown Toronto               0   
17                                       East Toronto               0   
20                                    Central Toronto               0   
22                                       West Toronto               0   
23                                    Central Toronto               0   
24                                    Central Toronto               0   
25                                       West Toronto               0   
26                                    Central Toronto               0   
27                                   Downtown Toronto               0   
28                                       West Toronto               0   
30                                   Downtown Toronto               0   
31                                    Central Toronto               0   
32                                   Downtown Toronto               0   
34     Downtown TorontoStn A PO Boxes25 The Esplanade               0   
35                                   Downtown Toronto               0   
36                                   Downtown Toronto               0   
37                                   Downtown Toronto               0   
38  East TorontoBusiness reply mail Processing Cen...               0   

   1st Most Common Venue 2nd Most Common Venue  3rd Most Common Venue  \
0            Coffee Shop                Bakery                   Park   
1         Clothing Store           Coffee Shop         Cosmetics Shop   
2                   Café           Coffee Shop           Cocktail Bar   
3      Health Food Store                 Trail                    Pub   
4            Coffee Shop                Bakery           Cocktail Bar   
5            Coffee Shop        Sandwich Place                   Café   
6          Grocery Store                  Café                   Park   
7            Coffee Shop                  Café             Restaurant   
8               Pharmacy                Bakery                    Bar   
10           Coffee Shop              Aquarium                   Café   
11                   Bar           Coffee Shop  Vietnamese Restaurant   
12      Greek Restaurant    Italian Restaurant            Coffee Shop   
13           Coffee Shop                 Hotel                   Café   
14                  Café        Breakfast Spot  Performing Arts Venue   
15                  Park  Fast Food Restaurant                    Gym   
16           Coffee Shop            Restaurant                   Café   
17           Coffee Shop               Brewery                   Café   
20                   Gym                 Hotel      Convenience Store   
22                  Café    Mexican Restaurant        Thai Restaurant   
23           Coffee Shop        Clothing Store            Yoga Studio   
24                  Café      

### Cluster 2

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
19  Central Toronto               1                Garden   

   2nd Most Common Venue 3rd Most Common Venue     4th Most Common Venue  \
19                Museum        Massage Studio  Mediterranean Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
19           Men's Store         Metro Station    Mexican Restaurant   

        8th Most Common Venue 9th Most Common Venue  \
19  Middle Eastern Restaurant    Miscellaneous Shop   

        10th Most Common Venue  
19  Modern European Restaurant

### Cluster 3

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
9   East YorkEast Toronto               2                  Park   
21        Central Toronto               2                  Park   
33       Downtown Toronto               2                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
9      Convenience Store         Metro Station           Yoga Studio   
21                 Trail         Jewelry Store      Sushi Restaurant   
33            Playground                 Trail           Yoga Studio   

   5th Most Common Venue     6th Most Common Venue     7th Most Common Venue  \
9                 Museum  Mediterranean Restaurant               Men's Store   
21           Yoga Studio             Movie Theater  Mediterranean Restaurant   
33                Museum  Mediterranean Restaurant               Men's Store   

   8th Most Common Venue      9th Most Common Venue     10th Most Common Venue  
9     Mexican Restaurant  Middle Eastern Restaurant         Miscellaneous Shop  
21           Men's Store              Metro Station         Mexican Restaurant  
33         Metro Station         Mexican Restaurant  Middle Eastern Restaurant

### Cluster 4

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
18  Central Toronto               3                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
18              Bus Line           Swim School           Yoga Studio   

   5th Most Common Venue     6th Most Common Venue 7th Most Common Venue  \
18         Movie Theater  Mediterranean Restaurant           Men's Store   

   8th Most Common Venue 9th Most Common Venue     10th Most Common Venue  
18         Metro Station    Mexican Restaurant  Middle Eastern Restaurant

### Cluster 5

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
29  Central Toronto               4                  Park   

   2nd Most Common Venue 3rd Most Common Venue     4th Most Common Venue  \
29           Yoga Studio                Museum  Mediterranean Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
29           Men's Store         Metro Station    Mexican Restaurant   

        8th Most Common Venue 9th Most Common Venue  \
29  Middle Eastern Restaurant    Miscellaneous Shop   

        10th Most Common Venue  
29  Modern European Restaurant